In [1]:
# === Genome3D v1h • Pro Viewer (chromosome colors, equal aspect, toggles) ===
# Loads v1h nodes/edges from the manifest and builds an upgraded Plotly HTML.
# Tags the output: ["genome3d","atlas","v1h","interactive","pro"]

import numpy as np, pandas as pd
from pathlib import Path
import cntlab as cl

cl.nb.init()

# -- helpers
def hits(kind,*tags): return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
def grab(kind,*tags): 
    H = hits(kind,*tags); assert H, f"No {kind} for tags={tags}"; 
    return H[-1]["path"]

# Load v1h nodes/edges
nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]
nodes = pd.read_parquet(nodes_pq)
edges = pd.read_parquet(edges_pq)

# Split genes vs anchors
genes = nodes[nodes["kind"].astype(str).str.lower()=="gene"].copy()
anchors = nodes[nodes["kind"].astype(str).str.lower()=="anchor"].copy()

# Color map per chromosome (stable, readable)
all_chrs = sorted(genes["chr"].unique(), key=lambda c: (0,int(c[3:])) if c[3:].isdigit() else (1, c))
palette = [
    "#4c78a8","#f58518","#54a24b","#e45756","#72b7b2","#b279a2",
    "#ff9da6","#9d755d","#bab0ab","#ff7f0e","#2ca02c","#d62728",
    "#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf",
    "#1f77b4","#aec7e8","#ffbb78","#98df8a","#ff9896","#c5b0d5","#c49c94"
]
color_by_chr = {ch: palette[i % len(palette)] for i,ch in enumerate(all_chrs)}

# Downsample genes only for plotting (data table stays full on disk already)
MAX_POINTS = 65000
if len(genes) > MAX_POINTS:
    genes_plot = genes.sample(MAX_POINTS, random_state=1337)
else:
    genes_plot = genes

# Optional constellation ring (connect chr midpoints)
CONSTELLATION = True
const_x = const_y = const_z = None
if CONSTELLATION:
    mids = []
    for ch, segs in edges.groupby("chr"):
        segs = segs.sort_values("seg")
        # mid segment
        mid = len(segs)//2
        mids.append((ch, float(segs.iloc[mid]["x0"]), float(segs.iloc[mid]["y0"]), float(segs.iloc[mid]["z0"])))
    mids = sorted(mids, key=lambda t: all_chrs.index(t[0]) if t[0] in all_chrs else 999)
    if mids:
        arr = np.array([[x,y,z] for _,x,y,z in mids])
        const_x, const_y, const_z = np.r_[arr[:,0], arr[0,0]], np.r_[arr[:,1], arr[0,1]], np.r_[arr[:,2], arr[0,2]]

# Build Plotly
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable,"-m","pip","install","plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()

# Backbones (colored per chromosome)
for ch, segs in edges.groupby("chr"):
    xs = np.column_stack([segs["x0"],segs["x1"]]).ravel()
    ys = np.column_stack([segs["y0"],segs["y1"]]).ravel()
    zs = np.column_stack([segs["z0"],segs["z1"]]).ravel()
    xs = np.insert(xs, np.arange(2,xs.size,2), np.nan)
    ys = np.insert(ys, np.arange(2,ys.size,2), np.nan)
    zs = np.insert(zs, np.arange(2,zs.size,2), np.nan)
    col = color_by_chr.get(ch, "rgba(80,80,80,0.7)")
    fig.add_trace(go.Scatter3d(
        x=xs,y=ys,z=zs, mode="lines",
        line=dict(width=2, color=col),
        name=f"{ch} backbone",
        hoverinfo="skip",
        visible=True
    ))

# Constellation ring (faint)
if CONSTELLATION and const_x is not None:
    fig.add_trace(go.Scatter3d(
        x=const_x, y=const_y, z=const_z, mode="lines",
        line=dict(width=1, color="rgba(80,80,80,0.25)", dash="dot"),
        name="constellation",
        hoverinfo="skip",
        visible=True
    ))

# Genes: one trace per chromosome for legend toggles + colors
for ch, g in genes_plot.groupby("chr"):
    fig.add_trace(go.Scatter3d(
        x=g["x"], y=g["y"], z=g["z"],
        mode="markers",
        marker=dict(size=2, color=color_by_chr.get(ch,"#666")),
        name=f"{ch} genes",
        customdata=np.stack([g.get("gene",""), g["chr"], g["pos"], g.get("type","")], axis=1),
        hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
        visible=True
    ))

# Layout polish
fig.update_layout(
    title="Genome3D v1h — Pro Viewer (Hi-C warped, colored by chromosome)",
    scene=dict(
        xaxis_title="x", yaxis_title="y", zaxis_title="z",
        aspectmode="data"  # equal aspect so distances look right
    ),
    legend=dict(itemsizing="constant"),
    height=820, width=1180, margin=dict(l=0,r=0,t=60,b=0)
)

# Layer toggles via updatemenus
buttons = [
    dict(label="Show All", method="update",
         args=[{"visible":[True]*len(fig.data)}]),
    dict(label="Backbones only", method="update",
         args=[{"visible":[True]* (len(edges["chr"].unique()) + (1 if CONSTELLATION else 0)) + [False]*len(genes_plot["chr"].unique())}]),
    dict(label="Genes only", method="update",
         args=[{"visible":[False]* (len(edges["chr"].unique()) + (1 if CONSTELLATION else 0)) + [True]*len(genes_plot["chr"].unique())}])
]
fig.update_layout(
    updatemenus=[dict(type="buttons", direction="left", x=0.0, y=1.1,
                      buttons=buttons, pad={"r":6,"t":6})]
)

# Save HTML
html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(
    html.encode("utf-8"),
    module="genome3d", dataset="atlas", desc="interactive_v1h_pro",
    tags=["genome3d","atlas","v1h","interactive","pro"], ext="html"
)

print("Interactive (Pro) →", html_path)


[2025-10-08 21:59:56,088] INFO cntlab: CNTLab notebook initialized
[2025-10-08 21:59:56,089] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics
Interactive (Pro) → C:\Users\caleb\CNT_Lab\artifacts\genome3d__atlas__interactive_v1h_pro__20251008-215956.html


In [2]:
# === Genome3D v1h • Pro+ Viewer (type filters + per-chr focus) ===============
# Loads v1h nodes/edges from the manifest and builds an advanced Plotly HTML:
#  - Mode toggle: color by Chromosome OR by Gene Type
#  - Per-chromosome focus dropdown (show only chrN)
#  - Equal aspect, thicker backbones, higher point cap
# Output tags: ["genome3d","atlas","v1h","interactive","pro+","filters"]

import numpy as np, pandas as pd
from pathlib import Path
import cntlab as cl

cl.nb.init()

# --- helpers
def hits(kind,*tags): 
    return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
def grab(kind,*tags):
    H = hits(kind,*tags)
    assert H, f"No {kind} for tags={tags}"
    return H[-1]["path"]

# Load v1h artifacts
nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]
nodes = pd.read_parquet(nodes_pq)
edges = pd.read_parquet(edges_pq)

# Split gene/anchor
nodes["kind"] = nodes["kind"].astype(str).str.lower()
genes  = nodes[nodes["kind"]=="gene"].copy()
anchors= nodes[nodes["kind"]=="anchor"].copy()

# Order chromosomes nicely
def chr_key(ch):
    tail = ch[3:]
    return (0, int(tail)) if tail.isdigit() else (1, {"X":23,"Y":24,"MT":25}.get(tail.upper(), 99))
chrs = sorted(genes["chr"].unique(), key=chr_key)

# Top gene types (cap to keep traces light)
type_counts = genes["type"].fillna("").replace("", "other").value_counts()
top_types = list(type_counts.index[:8])
genes["type_slim"] = genes["type"].fillna("").replace("", "other")
genes.loc[~genes["type_slim"].isin(top_types), "type_slim"] = "other"
types = sorted(genes["type_slim"].unique(), key=lambda t: (0, -type_counts.get(t,0)) if t!="other" else (1,0))

# Sample for plotting (data tables remain full on disk already)
MAX_POINTS = 90000
genes_plot = genes.sample(MAX_POINTS, random_state=1337) if len(genes)>MAX_POINTS else genes

# Palettes
chr_palette = [
    "#4c78a8","#f58518","#54a24b","#e45756","#72b7b2","#b279a2",
    "#ff9da6","#9d755d","#bab0ab","#ff7f0e","#2ca02c","#d62728",
    "#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf",
    "#1f77b4","#aec7e8","#ffbb78","#98df8a","#ff9896","#c5b0d5","#c49c94"
]
type_palette = [
    "#00876c","#6aaa96","#d3d3d3","#d4886a","#c43e3f","#5b5bff","#c06cdb","#ffa600"
]
color_by_chr  = {ch: chr_palette[i % len(chr_palette)] for i,ch in enumerate(chrs)}
color_by_type = {t: type_palette[i % len(type_palette)] for i,t in enumerate(types)}

# Build gene traces for two modes:
#  A) chromosome-colored: per-chr
#  B) type-colored: per-type (global)
chr_gene_traces = []
for ch, g in genes_plot.groupby("chr"):
    chr_gene_traces.append(dict(
        name=f"{ch} genes",
        x=g["x"].to_numpy(), y=g["y"].to_numpy(), z=g["z"].to_numpy(),
        color=color_by_chr.get(ch,"#666"),
        custom=np.stack([g["gene"], g["chr"], g["pos"], g["type"]], axis=1)
    ))

type_gene_traces = []
for t, g in genes_plot.groupby("type_slim"):
    type_gene_traces.append(dict(
        name=f"{t} genes",
        x=g["x"].to_numpy(), y=g["y"].to_numpy(), z=g["z"].to_numpy(),
        color=color_by_type.get(t,"#888"),
        custom=np.stack([g["gene"], g["chr"], g["pos"], g["type"]], axis=1)
    ))

# Build backbone traces per chr
bb_traces = []
for ch, segs in edges.groupby("chr"):
    xs = np.column_stack([segs["x0"].values, segs["x1"].values]).ravel(order="C")
    ys = np.column_stack([segs["y0"].values, segs["y1"].values]).ravel(order="C")
    zs = np.column_stack([segs["z0"].values, segs["z1"].values]).ravel(order="C")
    xs = np.insert(xs, np.arange(2, xs.size, 2), np.nan)
    ys = np.insert(ys, np.arange(2, ys.size, 2), np.nan)
    zs = np.insert(zs, np.arange(2, zs.size, 2), np.nan)
    bb_traces.append(dict(
        ch=ch, x=xs, y=ys, z=zs, color=color_by_chr.get(ch, "rgba(80,80,80,0.6)")
    ))

# --- Plotly
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable,"-m","pip","install","plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()

# Add backbone traces (first N traces)
for bb in bb_traces:
    fig.add_trace(go.Scatter3d(
        x=bb["x"], y=bb["y"], z=bb["z"], mode="lines",
        name=f"{bb['ch']} backbone",
        line=dict(width=2, color=bb["color"]),
        hoverinfo="skip", visible=True
    ))
n_bb = len(bb_traces)

# Add gene traces — chromosome-colored (active by default)
for tr in chr_gene_traces:
    fig.add_trace(go.Scatter3d(
        x=tr["x"], y=tr["y"], z=tr["z"], mode="markers",
        marker=dict(size=2, color=tr["color"]),
        name=tr["name"], customdata=tr["custom"],
        hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
        visible=True
    ))
n_chrgenes = len(chr_gene_traces)

# Add gene traces — type-colored (start hidden)
for tr in type_gene_traces:
    fig.add_trace(go.Scatter3d(
        x=tr["x"], y=tr["y"], z=tr["z"], mode="markers",
        marker=dict(size=2, color=tr["color"]),
        name=tr["name"], customdata=tr["custom"],
        hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
        visible=False
    ))
n_typegenes = len(type_gene_traces)

# Visibility helpers
def vis_all(mode="chr"):
    # mode: "chr" (chr-colored genes) or "type" (type-colored genes)
    vis = [True]*n_bb
    if mode=="chr":
        vis += [True]*n_chrgenes + [False]*n_typegenes
    else:
        vis += [False]*n_chrgenes + [True]*n_typegenes
    return vis

def vis_only_chr(target, mode="chr"):
    # show backbones only for target chr, and genes for that chr (chr-mode) or all genes (type-mode)
    vis = []
    # backbones
    for bb in bb_traces:
        vis.append(bb["ch"] == target)
    # chr gene traces
    if mode=="chr":
        for tr in chr_gene_traces:
            vis.append(tr["name"].startswith(target))
        vis += [False]*n_typegenes
    else:
        # show only genes of target in type-mode? keep all types but mask by chr (approx via legend is hard).
        # Simpler: show all type traces (they include all chrs) + only target backbone to focus.
        vis += [False]*n_chrgenes
        vis += [True]*n_typegenes
    return vis

# Buttons: color mode
buttons_mode = [
    dict(label="Color by Chromosome", method="update",
         args=[{"visible": vis_all("chr")},
               {"title": "Genome3D v1h — Chromosome mode"}]),
    dict(label="Color by Gene Type", method="update",
         args=[{"visible": vis_all("type")},
               {"title": "Genome3D v1h — Gene-type mode"}])
]

# Dropdown: per-chr focus (on current mode we default to chr-mode)
dropdown_chr = [dict(label="All chromosomes", method="update",
                     args=[{"visible": vis_all("chr")}])]
for ch in chrs:
    dropdown_chr.append(dict(label=f"Focus: {ch}", method="update",
                             args=[{"visible": vis_only_chr(ch, "chr")}]))
# Layout
fig.update_layout(
    title="Genome3D v1h — Pro+ (Hi-C warped, modes & filters)",
    scene=dict(xaxis_title="x", yaxis_title="y", zaxis_title="z", aspectmode="data"),
    height=840, width=1200, margin=dict(l=0,r=0,t=60,b=0),
    updatemenus=[
        dict(type="buttons", direction="left", x=0.0, y=1.12, buttons=buttons_mode, pad={"r":6,"t":6}),
        dict(type="dropdown", direction="down", x=0.62, y=1.12, buttons=dropdown_chr, pad={"r":6,"t":6})
    ],
    legend=dict(itemsizing="constant")
)

# Save HTML
html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(
    html.encode("utf-8"),
    module="genome3d", dataset="atlas", desc="interactive_v1h_pro_plus",
    tags=["genome3d","atlas","v1h","interactive","pro+","filters"], ext="html"
)
print("Interactive (Pro+) →", html_path)


[2025-10-08 22:05:41,218] INFO cntlab: CNTLab notebook initialized
[2025-10-08 22:05:41,219] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics
Interactive (Pro+) → C:\Users\caleb\CNT_Lab\artifacts\genome3d__atlas__interactive_v1h_pro_plus__20251008-220541.html


In [3]:
# === Genome3D v1h • Starfield Viewer =========================================
# Aesthetic goals:
# - near-black background, no grids/axes clutter
# - chromosomes as faint constellations
# - genes as glowing stars (two-layer halo+core)
# - subtle depth "dust" for parallax feel
#
# Output: HTML tagged ["genome3d","atlas","v1h","interactive","starfield"]

import numpy as np, pandas as pd
from pathlib import Path
import cntlab as cl

cl.nb.init()

# ---------- Load v1h nodes/edges from manifest ----------
def hits(kind,*tags): 
    return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
def grab(kind,*tags):
    H = hits(kind,*tags)
    assert H, f"No {kind} for tags={tags}"
    return H[-1]["path"]

nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]
nodes = pd.read_parquet(nodes_pq)
edges = pd.read_parquet(edges_pq)

genes = nodes[nodes["kind"].astype(str).str.lower()=="gene"].copy()

# ---------- Prep star colors (cool → warm “spectral” palette) ----------
# derive a stable pseudo-random color per gene from chr + pos (no perf hit)
key = (genes["chr"].astype(str) + "|" + genes["pos"].astype(str)).str.hash().astype("uint32")
t = (key % 1000) / 1000.0  # 0..1
# map t to temperature-ish RGB: blue-white → white → warm
def lerp(a,b,x): return a + (b-a)*x
blue = np.array([155,188,255]) / 255.0   # #9bbcff
white= np.array([255,255,255]) / 255.0   # #ffffff
warm = np.array([255,220,168]) / 255.0   # #ffdca8
rgb  = np.where(t.values.reshape(-1,1) < 0.55,
                lerp(blue, white, (t/0.55).values.reshape(-1,1)),
                lerp(white, warm, ((t-0.55)/0.45).values.reshape(-1,1)))
star_colors = ["rgb(%d,%d,%d)" % (int(255*c[0]), int(255*c[1]), int(255*c[2])) for c in rgb]

# ---------- Downsample for interactivity (data tables already saved full) ----------
MAX_POINTS = 120_000
if len(genes) > MAX_POINTS:
    sel = genes.sample(MAX_POINTS, random_state=1337).copy()
    sel_colors = np.array(star_colors)[sel.index]
else:
    sel = genes.copy()
    sel_colors = np.array(star_colors)

# ---------- Constellation lines (faint) ----------
# build continuous polylines per chromosome from edges
def backbone_lines(df):
    xs, ys, zs = [], [], []
    for ch, segs in df.groupby("chr"):
        segs = segs.sort_values("seg")
        x = np.column_stack([segs["x0"].values, segs["x1"].values]).ravel(order="C")
        y = np.column_stack([segs["y0"].values, segs["y1"].values]).ravel(order="C")
        z = np.column_stack([segs["z0"].values, segs["z1"].values]).ravel(order="C")
        # insert NaNs between segments so Plotly breaks lines
        x = np.insert(x, np.arange(2, x.size, 2), np.nan)
        y = np.insert(y, np.arange(2, y.size, 2), np.nan)
        z = np.insert(z, np.arange(2, z.size, 2), np.nan)
        xs.append(x); ys.append(y); zs.append(z)
    return xs, ys, zs

bx, by, bz = backbone_lines(edges)

# ---------- Star-dust layer (random faint points within atlas bounds) ----------
mins = nodes[["x","y","z"]].min()
maxs = nodes[["x","y","z"]].max()
rng = np.random.default_rng(1337)
N_DUST = 6000
dust = np.column_stack([
    rng.uniform(mins["x"], maxs["x"], N_DUST),
    rng.uniform(mins["y"], maxs["y"], N_DUST),
    rng.uniform(mins["z"], maxs["z"], N_DUST)
])

# ---------- Plotly scene ----------
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable, "-m", "pip", "install", "plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()

# 1) Constellations (faint, dashed)
for x,y,z in zip(bx,by,bz):
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z, mode="lines",
        line=dict(width=1.2, color="rgba(200,200,200,0.18)", dash="dot"),
        hoverinfo="skip", name="constellation", showlegend=False, visible=True
    ))

# 2) Dust (very faint)
fig.add_trace(go.Scatter3d(
    x=dust[:,0], y=dust[:,1], z=dust[:,2],
    mode="markers",
    marker=dict(size=1, color="rgba(200,200,255,0.05)"),
    name="dust", hoverinfo="skip", showlegend=False, visible=True
))

# 3) Stars (two layers: halo then core)
# Halo (bigger, low alpha)
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers",
    marker=dict(size=4, color="rgba(170,190,255,0.10)"),
    name="genes halo", hoverinfo="skip", showlegend=False, visible=True
))
# Core (smaller, bright, colored)
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers",
    marker=dict(size=1.8, color=sel_colors.tolist()),
    name="genes",
    customdata=np.stack([sel.get("gene",""), sel["chr"], sel["pos"], sel.get("type","")], axis=1),
    hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
    showlegend=True
))

# Aesthetic: deep background, no grids, equal aspect
bg = "#0a0d12"
axis_style = dict(showgrid=False, showline=False, zeroline=False,
                  ticks="", showbackground=False, color="rgba(255,255,255,0.25)")
fig.update_layout(
    title="<b>Genome3D v1h — Starfield</b>",
    scene=dict(
        bgcolor=bg,
        xaxis=axis_style, yaxis=axis_style, zaxis=axis_style,
        aspectmode="data"
    ),
    paper_bgcolor=bg,
    font=dict(color="rgba(255,255,255,0.85)"),
    legend=dict(itemsizing="constant", bgcolor="rgba(0,0,0,0)"),
    height=860, width=1220, margin=dict(l=0,r=0,t=50,b=0)
)

# Quick layer toggles
fig.update_layout(
    updatemenus=[dict(
        type="buttons", direction="left", x=0.02, y=1.08,
        buttons=[
            dict(label="All", method="update", args=[{"visible":[True]*len(fig.data)}]),
            dict(label="Stars only", method="update", args=[{"visible":[False]*(len(bx)+1)+[True,True]}]),
            dict(label="Constellations only", method="update", args=[{"visible":[True]*len(bx)+[False]*(len(fig.data)-len(bx))}]),
        ], pad={"r":6,"t":6}
    )]
)

# Save HTML
html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(
    html.encode("utf-8"),
    module="genome3d", dataset="atlas", desc="interactive_v1h_starfield",
    tags=["genome3d","atlas","v1h","interactive","starfield"], ext="html"
)
print("Starfield interactive →", html_path)


[2025-10-08 22:09:30,913] INFO cntlab: CNTLab notebook initialized
[2025-10-08 22:09:30,913] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics


AttributeError: 'StringMethods' object has no attribute 'hash'

In [4]:
# === Genome3D v1h • Starfield Viewer (fixed, single cell) ====================
# Deep-black scene, glowing stars, faint constellation lines, subtle dust.
# Output tag: ["genome3d","atlas","v1h","interactive","starfield"]

import numpy as np, pandas as pd
from pathlib import Path
import cntlab as cl

cl.nb.init()

# ---------- Load v1h nodes/edges ----------
def hits(kind,*tags): 
    return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
def grab(kind,*tags):
    H = hits(kind,*tags); assert H, f"No {kind} for tags={tags}"; return H[-1]["path"]

nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]
nodes = pd.read_parquet(nodes_pq)
edges = pd.read_parquet(edges_pq)
genes = nodes[nodes["kind"].astype(str).str.lower()=="gene"].copy()

# ---------- Stable per-gene color key ----------
from pandas.util import hash_pandas_object
sig = genes[["chr","pos"]].astype(str).agg("|".join, axis=1)
key = hash_pandas_object(sig, index=False).astype("uint64").to_numpy()
t = (key % 1000) / 1000.0  # 0..1

def lerp(a,b,x): return a + (b-a)*x
blue = np.array([155,188,255]) / 255.0   # cool
white= np.array([255,255,255]) / 255.0   # neutral
warm = np.array([255,220,168]) / 255.0   # warm
mix1 = lerp(blue,  white, np.clip(t/0.55, 0, 1))
mix2 = lerp(white, warm, np.clip((t-0.55)/0.45, 0, 1))
rgb  = np.where((t<0.55)[:,None], mix1, mix2)
star_colors = ["rgb(%d,%d,%d)" % (int(255*c[0]), int(255*c[1]), int(255*c[2])) for c in rgb]

# ---------- Downsample for interactivity ----------
MAX_POINTS = 120_000
if len(genes) > MAX_POINTS:
    sel = genes.sample(MAX_POINTS, random_state=1337).copy()
    sel_colors = np.array(star_colors)[sel.index]
else:
    sel = genes.copy()
    sel_colors = np.array(star_colors)

# ---------- Constellation polylines ----------
def backbone_lines(df):
    xs, ys, zs = [], [], []
    for ch, segs in df.groupby("chr"):
        segs = segs.sort_values("seg")
        x = np.column_stack([segs["x0"].values, segs["x1"].values]).ravel(order="C")
        y = np.column_stack([segs["y0"].values, segs["y1"].values]).ravel(order="C")
        z = np.column_stack([segs["z0"].values, segs["z1"].values]).ravel(order="C")
        x = np.insert(x, np.arange(2, x.size, 2), np.nan)
        y = np.insert(y, np.arange(2, y.size, 2), np.nan)
        z = np.insert(z, np.arange(2, z.size, 2), np.nan)
        xs.append(x); ys.append(y); zs.append(z)
    return xs, ys, zs
bx, by, bz = backbone_lines(edges)

# ---------- Star-dust layer ----------
mins = nodes[["x","y","z"]].min(); maxs = nodes[["x","y","z"]].max()
rng = np.random.default_rng(1337)
N_DUST = 6000
dust = np.column_stack([
    rng.uniform(mins["x"], maxs["x"], N_DUST),
    rng.uniform(mins["y"], maxs["y"], N_DUST),
    rng.uniform(mins["z"], maxs["z"], N_DUST)
])

# ---------- Plotly scene ----------
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable, "-m", "pip", "install", "plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()

# Constellations (faint, dashed)
for x,y,z in zip(bx,by,bz):
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z, mode="lines",
        line=dict(width=1.2, color="rgba(200,200,200,0.18)", dash="dot"),
        hoverinfo="skip", name="constellation", showlegend=False, visible=True
    ))

# Dust
fig.add_trace(go.Scatter3d(
    x=dust[:,0], y=dust[:,1], z=dust[:,2],
    mode="markers",
    marker=dict(size=1, color="rgba(200,200,255,0.05)"),
    name="dust", hoverinfo="skip", showlegend=False, visible=True
))

# Stars: halo + core
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers", marker=dict(size=4, color="rgba(170,190,255,0.10)"),
    name="genes halo", hoverinfo="skip", showlegend=False, visible=True
))
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers", marker=dict(size=1.8, color=sel_colors.tolist()),
    name="genes",
    customdata=np.stack([sel.get("gene",""), sel["chr"], sel["pos"], sel.get("type","")], axis=1),
    hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
    showlegend=True
))

# Aesthetic polish
bg = "#0a0d12"
axis_style = dict(showgrid=False, showline=False, zeroline=False, ticks="", showbackground=False,
                  color="rgba(255,255,255,0.25)")
fig.update_layout(
    title="<b>Genome3D v1h — Starfield</b>",
    scene=dict(bgcolor=bg, xaxis=axis_style, yaxis=axis_style, zaxis=axis_style, aspectmode="data"),
    paper_bgcolor=bg, font=dict(color="rgba(255,255,255,0.85)"),
    legend=dict(itemsizing="constant", bgcolor="rgba(0,0,0,0)"),
    height=860, width=1220, margin=dict(l=0,r=0,t=50,b=0),
    updatemenus=[dict(
        type="buttons", direction="left", x=0.02, y=1.08,
        buttons=[
            dict(label="All", method="update", args=[{"visible":[True]*len(fig.data)}]),
            dict(label="Stars only", method="update", args=[{"visible":[False]*(len(bx)+1)+[True,True]}]),
            dict(label="Constellations only", method="update", args=[{"visible":[True]*len(bx)+[False]*(len(fig.data)-len(bx))}]),
        ], pad={"r":6,"t":6}
    )]
)

html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(
    html.encode("utf-8"),
    module="genome3d", dataset="atlas", desc="interactive_v1h_starfield",
    tags=["genome3d","atlas","v1h","interactive","starfield"], ext="html"
)
print("Starfield interactive →", html_path)


[2025-10-08 22:13:40,968] INFO cntlab: CNTLab notebook initialized
[2025-10-08 22:13:40,968] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics


ValueError: operands could not be broadcast together with shapes (3,) (42764,) 

In [5]:
# === Genome3D v1h • Starfield Viewer (broadcast-safe, single cell) ===========
import numpy as np, pandas as pd
from pathlib import Path
import cntlab as cl

cl.nb.init()

# --- Load v1h nodes/edges from manifest ---
def hits(kind,*tags): 
    return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
def grab(kind,*tags):
    H = hits(kind,*tags); assert H, f"No {kind} for tags={tags}"; return H[-1]["path"]

nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]
nodes = pd.read_parquet(nodes_pq)
edges = pd.read_parquet(edges_pq)
genes = nodes[nodes["kind"].astype(str).str.lower()=="gene"].copy()

# --- Stable per-gene color key → cool→white→warm gradient (broadcast-safe) ---
from pandas.util import hash_pandas_object
sig = genes[["chr","pos"]].astype(str).agg("|".join, axis=1)
key = hash_pandas_object(sig, index=False).astype("uint64").to_numpy()
t = (key % 1000) / 1000.0                                     # (N,)
t1 = np.clip(t/0.55, 0, 1).reshape(-1,1)                       # (N,1)
t2 = np.clip((t-0.55)/0.45, 0, 1).reshape(-1,1)                # (N,1)

blue  = np.array([[155,188,255]], dtype=float) / 255.0         # (1,3)
white = np.array([[255,255,255]], dtype=float) / 255.0         # (1,3)
warm  = np.array([[255,220,168]], dtype=float) / 255.0         # (1,3)

mix1 = blue  + (white - blue ) * t1                            # (N,3)
mix2 = white + (warm  - white) * t2                            # (N,3)
rgb  = np.where((t<0.55)[:,None], mix1, mix2)                  # (N,3)

def rgb_to_plotly(arr):  # arr: (N,3) in 0..1
    c = (np.clip(arr,0,1)*255).astype(int)
    return [f"rgb({r},{g},{b})" for r,g,b in c]

colors_all = np.array(rgb_to_plotly(rgb))                      # (N,)

# --- Downsample for interactivity; keep color alignment by index ---
MAX_POINTS = 120_000
if len(genes) > MAX_POINTS:
    sel = genes.sample(MAX_POINTS, random_state=1337).copy()
else:
    sel = genes.copy()
sel_colors = colors_all[sel.index.to_numpy()]                  # (n_sel,)

# --- Constellation polylines from edges ---
def backbone_lines(df):
    xs, ys, zs = [], [], []
    for _, segs in df.groupby("chr"):
        segs = segs.sort_values("seg")
        x = np.column_stack([segs["x0"].to_numpy(), segs["x1"].to_numpy()]).ravel(order="C")
        y = np.column_stack([segs["y0"].to_numpy(), segs["y1"].to_numpy()]).ravel(order="C")
        z = np.column_stack([segs["z0"].to_numpy(), segs["z1"].to_numpy()]).ravel(order="C")
        x = np.insert(x, np.arange(2, x.size, 2), np.nan)
        y = np.insert(y, np.arange(2, y.size, 2), np.nan)
        z = np.insert(z, np.arange(2, z.size, 2), np.nan)
        xs.append(x); ys.append(y); zs.append(z)
    return xs, ys, zs

bx, by, bz = backbone_lines(edges)

# --- Dust layer within atlas bounds ---
mins = nodes[["x","y","z"]].min(); maxs = nodes[["x","y","z"]].max()
rng = np.random.default_rng(1337)
N_DUST = 6000
dust = np.column_stack([
    rng.uniform(mins["x"], maxs["x"], N_DUST),
    rng.uniform(mins["y"], maxs["y"], N_DUST),
    rng.uniform(mins["z"], maxs["z"], N_DUST)
])

# --- Plotly scene ---
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable, "-m", "pip", "install", "plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()

# Constellations (faint)
for x,y,z in zip(bx,by,bz):
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z, mode="lines",
        line=dict(width=1.2, color="rgba(200,200,200,0.18)", dash="dot"),
        hoverinfo="skip", name="constellation", showlegend=False
    ))

# Dust
fig.add_trace(go.Scatter3d(
    x=dust[:,0], y=dust[:,1], z=dust[:,2],
    mode="markers", marker=dict(size=1, color="rgba(200,200,255,0.05)"),
    name="dust", hoverinfo="skip", showlegend=False
))

# Stars: halo + core (colors aligned)
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers", marker=dict(size=4, color="rgba(170,190,255,0.10)"),
    name="genes halo", hoverinfo="skip", showlegend=False
))
fig.add_trace(go.Scatter3d(
    x=sel["x"], y=sel["y"], z=sel["z"],
    mode="markers", marker=dict(size=1.8, color=sel_colors.tolist()),
    name="genes",
    customdata=np.stack([sel.get("gene",""), sel["chr"], sel["pos"], sel.get("type","")], axis=1),
    hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}",
    showlegend=True
))

# Aesthetics
bg = "#0a0d12"
axis_style = dict(showgrid=False, showline=False, zeroline=False, ticks="", showbackground=False,
                  color="rgba(255,255,255,0.25)")
fig.update_layout(
    title="<b>Genome3D v1h — Starfield</b>",
    scene=dict(bgcolor=bg, xaxis=axis_style, yaxis=axis_style, zaxis=axis_style, aspectmode="data"),
    paper_bgcolor=bg, font=dict(color="rgba(255,255,255,0.85)"),
    legend=dict(itemsizing="constant", bgcolor="rgba(0,0,0,0)"),
    height=860, width=1220, margin=dict(l=0,r=0,t=50,b=0),
    updatemenus=[dict(
        type="buttons", direction="left", x=0.02, y=1.08, pad={"r":6,"t":6},
        buttons=[
            dict(label="All", method="update", args=[{"visible":[True]*len(fig.data)}]),
            dict(label="Stars only", method="update", args=[{"visible":[False]*(len(bx)+1)+[True,True]}]),
            dict(label="Constellations only", method="update", args=[{"visible":[True]*len(bx)+[False]*(len(fig.data)-len(bx))}]),
        ]
    )]
)

html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(
    html.encode("utf-8"),
    module="genome3d", dataset="atlas", desc="interactive_v1h_starfield",
    tags=["genome3d","atlas","v1h","interactive","starfield"], ext="html"
)
print("Starfield interactive →", html_path)


[2025-10-08 22:18:44,874] INFO cntlab: CNTLab notebook initialized
[2025-10-08 22:18:44,875] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics
Starfield interactive → C:\Users\caleb\CNT_Lab\artifacts\genome3d__atlas__interactive_v1h_starfield__20251008-221846.html


In [6]:
# === Genome3D v1s — Starspread (real genes, no dust) =========================
# Spread each gene off its chromosome backbone using a local T/N/B frame.
# Saves:
#   tables: genome3d__atlas__nodes_v1s.parquet   ["genome3d","atlas","v1s"]
#   figure: genome3d__atlas__figure_v1s.png      ["genome3d","atlas","v1s"]
#   html:   genome3d__atlas__interactive_v1s.html["genome3d","atlas","v1s","interactive"]
# Inputs: v1h nodes/edges (from manifest). No fake points—only your genes.

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
import cntlab as cl

cl.nb.init(); P = cl.P

# ---- Tuning knobs ------------------------------------------------------------
R_PERP   = 0.35   # mean radial offset (in scene units) from backbone into N/B plane
R_JITTER = 0.20   # additional random jitter fraction
SEED     = 1337   # RNG seed for reproducibility
THICK_BACKBONES = 2
FAINT_ALPHA     = 0.25

# ---- Load v1h nodes + edges --------------------------------------------------
def hits(kind,*tags): return cl.manifest.find_artifacts(kind=kind, tags_all=list(tags))
nodes_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "nodes" in Path(h["path"]).name][-1]
edges_pq = [h["path"] for h in hits("table","genome3d","atlas","v1h") if "edges" in Path(h["path"]).name][-1]

nodes_v1h = pd.read_parquet(nodes_pq)
edges_v1h = pd.read_parquet(edges_pq)

genes = nodes_v1h[nodes_v1h["kind"].astype(str).str.lower()=="gene"].copy()
anchors = nodes_v1h[nodes_v1h["kind"].astype(str).str.lower()=="anchor"].copy()

# Rebuild per-chromosome polylines
polylines = {}
for ch, segs in edges_v1h.groupby("chr"):
    segs = segs.sort_values("seg")
    pts = np.vstack([segs[["x0","y0","z0"]].to_numpy(),
                     segs[["x1","y1","z1"]].to_numpy()[-1:]])
    polylines[ch] = pts

# ---- Utilities: interpolate & local frame (T/N/B) ----------------------------
def sample_polyline(P, s):
    """P: (K,3) points; s in [0,1] -> point on polyline by index interpolation"""
    s = np.clip(s, 0.0, 1.0)
    x = s*(len(P)-1)
    i0 = np.floor(x).astype(int)
    i1 = np.clip(i0+1, 0, len(P)-1)
    t  = (x - i0).reshape(-1,1)
    return (1-t)*P[i0] + t*P[i1]

def tangent_polyline(P, s):
    """Approximate tangent using neighboring samples."""
    eps = 1e-3
    a = sample_polyline(P, np.clip(s-eps,0,1))
    b = sample_polyline(P, np.clip(s+eps,0,1))
    v = (b - a)
    nrm = np.linalg.norm(v, axis=1, keepdims=True)+1e-12
    return v / nrm

def frame_TNB(P, s):
    """Construct a stable T/N/B frame at parameter s on polyline P."""
    T = tangent_polyline(P, s)                              # (n,3)
    # choose a reference vector not parallel to T
    ref = np.tile(np.array([[0.12,0.97,0.21]]), (len(T),1))
    # if nearly parallel, switch ref per-row
    dot = np.abs((T*ref).sum(axis=1))
    swap = dot > 0.95
    if swap.any():
        ref[swap] = np.array([0.35,0.05,0.93])
    N = np.cross(T, ref)
    N /= (np.linalg.norm(N, axis=1, keepdims=True)+1e-12)
    B = np.cross(T, N)
    B /= (np.linalg.norm(B, axis=1, keepdims=True)+1e-12)
    return T, N, B

# ---- Starspread: offset genes off the backbone --------------------------------
rng = np.random.default_rng(SEED)

# Normalize per-chromosome bp range using anchors if available; fallback by pos max
bp_max_by_chr = genes.groupby("chr")["pos"].max().to_dict()
if not anchors.empty:
    for ch, grp in anchors.groupby("chr"):
        # use 'end' anchor pos if present
        endpos = grp[grp["id"].astype(str).str.endswith(":end")]["pos"]
        if len(endpos):
            bp_max_by_chr[ch] = int(endpos.iloc[0])

# Map genes
star_rows = []
for ch, g in genes.groupby("chr", sort=False):
    if ch not in polylines: 
        continue
    P = polylines[ch]
    Lbp = max(1, bp_max_by_chr.get(ch, g["pos"].max()))
    # normalized parameter s in [0,1]
    s = (g["pos"].to_numpy() / Lbp).reshape(-1,1)
    s = np.clip(s, 0, 1).flatten()

    # base point on backbone and local frame
    base = sample_polyline(P, s)         # (n,3)
    T, N, B = frame_TNB(P, s)            # (n,3) each

    # star offsets: radius ~ R_PERP*(1 + jitter), random angle in N/B plane
    angle = rng.uniform(0, 2*np.pi, size=len(s))
    jitter = 1.0 + R_JITTER*rng.normal(0, 0.5, size=len(s))  # mild jitter
    radius = np.maximum(0.0, R_PERP * jitter)

    # compose offset
    offs = (N * np.cos(angle).reshape(-1,1) + B * np.sin(angle).reshape(-1,1)) * radius.reshape(-1,1)
    pos3 = base + offs

    part = pd.DataFrame({
        "id":   g.apply(lambda r: f"{(r['gene'] or 'id')}|{ch}|{int(r['pos'])}", axis=1).values,
        "chr":  ch,
        "kind": "gene",
        "pos":  g["pos"].astype(int).values,
        "x":    pos3[:,0], "y": pos3[:,1], "z": pos3[:,2],
        "gene": g.get("gene","").values,
        "type": g.get("type","").values
    })
    star_rows.append(part)

nodes_v1s = pd.concat(star_rows, ignore_index=True)

# Reuse original edges for faint constellations
edges_v1s = edges_v1h.copy()

# ---- Save v1s tables + snapshot ---------------------------------------------
nodes_path = cl.io.save_df(nodes_v1s, module="genome3d", dataset="atlas", desc="nodes_v1s", fmt="parquet",
                           tags=["genome3d","atlas","v1s"])
edges_path = cl.io.save_df(edges_v1s, module="genome3d", dataset="atlas", desc="edges_v1s", fmt="parquet",
                           tags=["genome3d","atlas","v1s"])

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')
# faint constellations
for ch, segs in edges_v1s.groupby("chr"):
    xs = np.column_stack([segs["x0"].values, segs["x1"].values]).ravel(order="C")
    ys = np.column_stack([segs["y0"].values, segs["y1"].values]).ravel(order="C")
    zs = np.column_stack([segs["z0"].values, segs["z1"].values]).ravel(order="C")
    xs = np.insert(xs, np.arange(2,xs.size,2), np.nan)
    ys = np.insert(ys, np.arange(2,ys.size,2), np.nan)
    zs = np.insert(zs, np.arange(2,zs.size,2), np.nan)
    ax.plot(xs,ys,zs, color="k", alpha=FAINT_ALPHA, linewidth=THICK_BACKBONES)
# stars
sample = nodes_v1s.sample(min(15000, len(nodes_v1s)), random_state=SEED)
ax.scatter(sample["x"], sample["y"], sample["z"], s=2, alpha=0.7, c="white")
ax.set_title("CNT Genome3D v1s — Starspread (real genes)")
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.view_init(elev=18, azim=25)
fig_path = cl.io.save_figure(fig, module="genome3d", dataset="atlas", desc="figure_v1s",
                             tags=["genome3d","atlas","v1s"])
plt.close(fig)

print("== v1s Starspread built ==")
print("Nodes  →", nodes_path)
print("Edges  →", edges_path)
print("Figure →", fig_path)

# ---- Interactive: minimal starfield (genes only + faint constellations) -----
try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    import sys, subprocess; subprocess.check_call([sys.executable,"-m","pip","install","plotly>=5.24"])
    import plotly.graph_objects as go

fig = go.Figure()
# constellations
for ch, segs in edges_v1s.groupby("chr"):
    xs = np.column_stack([segs["x0"], segs["x1"]]).ravel()
    ys = np.column_stack([segs["y0"], segs["y1"]]).ravel()
    zs = np.column_stack([segs["z0"], segs["z1"]]).ravel()
    xs = np.insert(xs, np.arange(2,xs.size,2), np.nan)
    ys = np.insert(ys, np.arange(2,ys.size,2), np.nan)
    zs = np.insert(zs, np.arange(2,zs.size,2), np.nan)
    fig.add_trace(go.Scatter3d(x=xs,y=ys,z=zs,mode="lines",
        line=dict(width=THICK_BACKBONES, color="rgba(255,255,255,0.20)"),
        name=f"{ch} constellation", hoverinfo="skip", showlegend=False))

# stars (genes)
MAX_POINTS = 120_000
stars = nodes_v1s if len(nodes_v1s) <= MAX_POINTS else nodes_v1s.sample(MAX_POINTS, random_state=SEED)
fig.add_trace(go.Scatter3d(
    x=stars["x"], y=stars["y"], z=stars["z"], mode="markers",
    marker=dict(size=1.8, color="rgba(255,255,255,0.95)"),
    name="genes",
    customdata=np.stack([stars.get("gene",""), stars["chr"], stars["pos"], stars.get("type","")], axis=1),
    hovertemplate="gene: %{customdata[0]}<br>chr: %{customdata[1]}  pos: %{customdata[2]}<br>type: %{customdata[3]}"
))

# scene polish
bg = "#05070b"
axis = dict(showgrid=False, showline=False, zeroline=False, ticks="", showbackground=False, color="rgba(255,255,255,0.25)")
fig.update_layout(
    title="<b>Genome3D v1s — Starspread</b>",
    scene=dict(bgcolor=bg, xaxis=axis, yaxis=axis, zaxis=axis, aspectmode="data"),
    paper_bgcolor=bg, font=dict(color="rgba(255,255,255,0.88)"),
    height=860, width=1220, margin=dict(l=0,r=0,t=50,b=0)
)

html = fig.to_html(include_plotlyjs="cdn", full_html=True)
html_path = cl.io.save_bytes(html.encode("utf-8"),
                             module="genome3d", dataset="atlas", desc="interactive_v1s_starspread",
                             tags=["genome3d","atlas","v1s","interactive"], ext="html")
print("Interactive →", html_path)


[2025-10-08 22:23:24,018] INFO cntlab: CNTLab notebook initialized
[2025-10-08 22:23:24,019] INFO cntlab: CNT Paths(root=C:\Users\caleb\CNT_Lab)


→ CNTLab ready.
   Root: C:\Users\caleb\CNT_Lab
   Figures: C:\Users\caleb\CNT_Lab\artifacts\figures
   Tables: C:\Users\caleb\CNT_Lab\artifacts\tables
   Metrics: C:\Users\caleb\CNT_Lab\artifacts\metrics
== v1s Starspread built ==
Nodes  → C:\Users\caleb\CNT_Lab\artifacts\tables\genome3d__atlas__nodes_v1s__20251008-222324.parquet
Edges  → C:\Users\caleb\CNT_Lab\artifacts\tables\genome3d__atlas__edges_v1s__20251008-222324.parquet
Figure → C:\Users\caleb\CNT_Lab\artifacts\figures\genome3d__atlas__figure_v1s__20251008-222324.png
Interactive → C:\Users\caleb\CNT_Lab\artifacts\genome3d__atlas__interactive_v1s_starspread__20251008-222324.html
